In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates

✅ Recommended Layouts
### 1. Single-Ticker Deep Dive (Your Current Layout)
- 	Keep as-is for focused analysis.
- 	Ideal for understanding one asset’s behavior across multiple dimensions.
### 2. Multi-Ticker Comparative Dashboard

🔹 Trend Indicators Panel
- 	Overlay Close Price, MA, EMA for all tickers.
- 	Normalize prices (e.g. rebased to 100) to compare trajectory.
- 	Use line plots with clear legends.

🔹 Momentum Panel
- 	RSI: overlay all tickers on one axis.
- 	Add horizontal lines at 30/70.
- 	Use consistent color mapping per ticker.

🔹 MACD Panel
- 	Plot MACD only, omit Signal Line to reduce clutter.
- 	Use subplots if more than 3 tickers.

🔹 Volatility Panel
- 	ATR: normalize per ticker or use separate y-axes.
- 	Alternatively, facet into small multiples (one ATR plot per ticker).

🔹 Range-Based Panel
- 	Stochastic Oscillator (%K, %D): hard to overlay cleanly.
- 	Best shown as faceted subplots per ticker.

🔹 Bollinger Bands
- 	Facet per ticker: Close, Upper Band, Lower Band.
- 	Overlaying multiple bands is visually noisy and misleading.

🔹 Volume Panel
- 	Normalize volume per ticker (e.g. divide by max).
- 	Overlay on shared axis or facet if absolute values matter.

In [ ]:
def fetch_yfinance_data(ticker, start_date, end_date, auto_adjust=True):
    return yf.download(ticker, start=start_date, end=end_date, auto_adjust=auto_adjust)

In [ ]:
def calculate_ma(data, window=20):
    """
    Calculate the moving average (MA) of the 'Close' price over a specified window.

    Parameters
    ----------
    data : pandas.DataFrame
        A DataFrame containing at least a 'Close' column with historical price data.
    window : int, optional (default=20)
        The number of periods to use for the moving average calculation.

    Returns
    -------
    pandas.DataFrame
        The original DataFrame with an additional 'MA' column containing the moving average values.

    Notes
    -----
    - The result will contain NaN values for the first (window - 1) rows.
    - This function modifies the input DataFrame in place.

    Example
    -------
    >>> df = yf.download("AAPL", start="2023-01-01", end="2024-01-01")
    >>> df = calculate_ma(df, window=50)
    >>> df[['Close', 'MA']].tail()
    """
    data['MA'] = data['Close'].rolling(window=window).mean()
    return data

In [ ]:
def calculate_ema(data, span=20):
    """
    Calculate the Exponential Moving Average (EMA) of the 'Close' price over a specified span.

    Parameters
    ----------
    data : pandas.DataFrame
        A DataFrame containing at least a 'Close' column with historical price data.
    span : int, optional (default=20)
        The span (in periods) for the EMA calculation. A smaller span gives more weight to recent prices.

    Returns
    -------
    pandas.DataFrame
        The original DataFrame with an additional 'EMA' column containing the exponential moving average values.

    Notes
    -----
    - EMA reacts more quickly to recent price changes than a simple moving average.
    - This function modifies the input DataFrame in place.

    Example
    -------
    >>> df = yf.download("MSFT", start="2023-01-01", end="2024-01-01")
    >>> df = calculate_ema(df, span=50)
    >>> df[['Close', 'EMA']].tail()
    """
    data['EMA'] = data['Close'].ewm(span=span, adjust=False).mean()
    return data

In [ ]:
def calculate_rsi(data, window=14):
    """
    Calculate the Relative Strength Index (RSI) for a given price series.

    Parameters
    ----------
    data : pandas.DataFrame
        A DataFrame containing at least a 'Close' column with historical price data.
    window : int, optional (default=14)
        The number of periods to use for calculating the average gains and losses.

    Returns
    -------
    pandas.DataFrame
        The original DataFrame with an additional 'RSI' column containing the Relative Strength Index values.

    Notes
    -----
    - RSI is a momentum oscillator that measures the speed and change of price movements.
    - Values range from 0 to 100. Traditionally, RSI > 70 indicates overbought, and RSI < 30 indicates oversold.
    - The first (window - 1) values will be NaN due to rolling calculations.
    - This function modifies the input DataFrame in place.

    Example
    -------
    >>> df = yf.download("TSLA", start="2023-01-01", end="2024-01-01")
    >>> df = calculate_rsi(df, window=14)
    >>> df[['Close', 'RSI']].tail()
    """
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))
    return data

In [ ]:
def calculate_macd(data, span_short=12, span_long=26, span_signal=9):
    """
    Calculate the Moving Average Convergence Divergence (MACD) and Signal Line for a given price series.

    Parameters
    ----------
    data : pandas.DataFrame
        A DataFrame containing at least a 'Close' column with historical price data.
    span_short : int, optional (default=12)
        The span for the short-term exponential moving average (EMA).
    span_long : int, optional (default=26)
        The span for the long-term exponential moving average (EMA).
    span_signal : int, optional (default=9)
        The span for the signal line, which is the EMA of the MACD line.

    Returns
    -------
    pandas.DataFrame
        The original DataFrame with two additional columns:
        - 'MACD': the difference between the short and long EMAs
        - 'Signal Line': the EMA of the MACD line

    Notes
    -----
    - MACD is a trend-following momentum indicator that shows the relationship between two EMAs.
    - The Signal Line is used to identify potential buy/sell signals when it crosses the MACD line.
    - This function modifies the input DataFrame in place.

    Example
    -------
    >>> df = yf.download("GOOG", start="2023-01-01", end="2024-01-01")
    >>> df = calculate_macd(df)
    >>> df[['Close', 'MACD', 'Signal Line']].tail()
    """
    data['MACD'] = data['Close'].ewm(span=span_short, adjust=False).mean() - data['Close'].ewm(span=span_long, adjust=False).mean()
    data['Signal Line'] = data['MACD'].ewm(span=span_signal, adjust=False).mean()
    return data

In [ ]:
# You may want to refactor this to compute the moving average internally (instead of requiring 'MA' to be precomputed), 
# or add plotting utilities for visualizing the bands.

def calculate_bollinger_bands(data, window=20):
    """
    Calculate Bollinger Bands for a given price series using a moving average and rolling standard deviation.

    Parameters
    ----------
    data : pandas.DataFrame
        A DataFrame containing at least a 'Close' column and a precomputed 'MA' (moving average) column.
    window : int, optional (default=20)
        The number of periods to use for the rolling standard deviation.

    Returns
    -------
    pandas.DataFrame
        The original DataFrame with three additional columns:
        - 'Rolling_STD': the rolling standard deviation of the 'Close' price
        - 'Upper Band': the upper Bollinger Band (MA + 2 × STD)
        - 'Lower Band': the lower Bollinger Band (MA - 2 × STD)

    Notes
    -----
    - This function assumes that a moving average ('MA') column already exists in the DataFrame.
    - Bollinger Bands are used to measure volatility and identify potential overbought or oversold conditions.
    - The bands widen during periods of high volatility and contract during periods of low volatility.
    - This function modifies the input DataFrame in place.

    Example
    -------
    >>> df = yf.download("NFLX", start="2023-01-01", end="2024-01-01")
    >>> df = calculate_ma(df, window=20)  # Required before calculating bands
    >>> df = calculate_bollinger_bands(df, window=20)
    >>> df[['Close', 'MA', 'Upper Band', 'Lower Band']].tail()
    """
    data['Rolling_STD'] = data['Close'].rolling(window=window).std()
    data['Upper Band'] = data['MA'] + (2 * data['Rolling_STD'])
    data['Lower Band'] = data['MA'] - (2 * data['Rolling_STD'])
    return data

In [ ]:
def calculate_stochastic_oscillator(data, window=14):
    """
    Calculate the Stochastic Oscillator (%K and %D) for a given price series.

    Parameters
    ----------
    data : pandas.DataFrame
        A DataFrame containing at least 'High', 'Low', and 'Close' columns with historical price data.
    window : int, optional (default=14)
        The number of periods to use for calculating the lowest low and highest high.

    Returns
    -------
    pandas.DataFrame
        The original DataFrame with two additional columns:
        - '%K': the current close relative to the recent high-low range
        - '%D': the 3-period simple moving average of %K (signal line)

    Notes
    -----
    - The Stochastic Oscillator is a momentum indicator comparing a security's closing price to its price range over a given period.
    - Values range from 0 to 100. Traditionally, %K > 80 indicates overbought, and %K < 20 indicates oversold.
    - This function modifies the input DataFrame in place.

    Example
    -------
    >>> df = yf.download("NVDA", start="2023-01-01", end="2024-01-01")
    >>> df = calculate_stochastic_oscillator(df, window=14)
    >>> df[['Close', '%K', '%D']].tail()
    """
    low_min = data['Low'].rolling(window=window).min()
    high_max = data['High'].rolling(window=window).max()
    data['%K'] = (data['Close'] - low_min) * 100 / (high_max - low_min)
    data['%D'] = data['%K'].rolling(window=3).mean()
    return data

In [ ]:
def calculate_atr(data, window=14):
    """
    Calculate the Average True Range (ATR), a measure of market volatility.

    Parameters
    ----------
    data : pandas.DataFrame
        A DataFrame containing at least 'High', 'Low', and 'Close' columns with historical price data.
    window : int, optional (default=14)
        The number of periods to use for the rolling average of the True Range.

    Returns
    -------
    pandas.DataFrame
        The original DataFrame with an additional 'ATR' column containing the Average True Range values.

    Notes
    -----
    - The True Range (TR) is defined as the maximum of:
        1. High - Low
        2. |High - Previous Close|
        3. |Low - Previous Close|
    - ATR is the rolling mean of the TR over the specified window.
    - ATR is commonly used to assess volatility and set stop-loss levels.
    - This function modifies the input DataFrame in place.

    Example
    -------
    >>> df = yf.download("SPY", start="2023-01-01", end="2024-01-01")
    >>> df = calculate_atr(df, window=14)
    >>> df[['High', 'Low', 'Close', 'ATR']].tail()
    """
    high_low = data['High'] - data['Low']
    high_close = np.abs(data['High'] - data['Close'].shift(1))
    low_close = np.abs(data['Low'] - data['Close'].shift(1))
    tr = np.maximum(high_low, high_close)
    tr = np.maximum(tr, low_close)
    data['ATR'] = tr.rolling(window=window).mean()
    return data

In [ ]:
def calculate_obv(data):
    """
    Calculate the On-Balance Volume (OBV) indicator for a given price and volume series.

    Parameters
    ----------
    data : pandas.DataFrame
        A DataFrame containing at least 'Close' and 'Volume' columns with historical price and volume data.

    Returns
    -------
    pandas.DataFrame
        The original DataFrame with an additional 'OBV' column containing the On-Balance Volume values.

    Notes
    -----
    - OBV is a cumulative volume-based momentum indicator.
    - Volume is added to OBV when the closing price increases, and subtracted when it decreases.
    - The first OBV value is initialized to 0.
    - This function modifies the input DataFrame in place.

    Example
    -------
    >>> df = yf.download("AMZN", start="2023-01-01", end="2024-01-01")
    >>> df = calculate_obv(df)
    >>> df[['Close', 'Volume', 'OBV']].tail()
    """
    obv = [0]  # Starting OBV value
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i - 1]:
            obv.append(obv[-1] + data['Volume'][i])
        elif data['Close'][i] < data['Close'][i - 1]:
            obv.append(obv[-1] - data['Volume'][i])
        else:
            obv.append(obv[-1])
    data['OBV'] = obv
    return data

In [ ]:
# deprecated due to warnings and lack of progess debugging with CoPilot 
# This version is especially useful when working with filtered or reindexed DataFrames, 
# ensuring robustness in production pipelines. 
# Implemented a vectorized version which avoids errors and warnings and for better performance 
# consider a wrapper to toggle between guarded and fast implementations if fixing it succeeds in the future.
def calculate_obv_guard(data):
    """
    Calculate the On-Balance Volume (OBV) indicator with index alignment safeguards.

    Parameters
    ----------
    data : pandas.DataFrame
        A DataFrame containing at least 'Close' and 'Volume' columns with historical price and volume data.

    Returns
    -------
    pandas.DataFrame
        The original DataFrame with an additional 'OBV' column containing the On-Balance Volume values.

    Notes
    -----
    - OBV is a cumulative volume-based momentum indicator.
    - Volume is added to OBV when the closing price increases, and subtracted when it decreases.
    - This implementation uses `.iloc` for safe row access and explicitly aligns the resulting OBV series with the DataFrame index.
    - The first OBV value is initialized to 0.
    - This function modifies the input DataFrame in place.

    Example
    -------
    >>> df = yf.download("META", start="2023-01-01", end="2024-01-01")
    >>> df = calculate_obv_guard(df)
    >>> df[['Close', 'Volume', 'OBV']].tail()
    """
    # Ensure 'Close' and 'Volume' are Series
    if isinstance(data['Close'], pd.DataFrame):
        data['Close'] = data['Close'].squeeze()
    if isinstance(data['Volume'], pd.DataFrame):
        data['Volume'] = data['Volume'].squeeze()

    obv = [0]
    for i in range(1, len(data)):
        current_close = data['Close'].iloc[i]
        previous_close = data['Close'].iloc[i - 1]
        current_volume = data['Volume'].iloc[i]

        # Ensure scalar comparison
        if float(current_close) > float(previous_close):
            obv.append(obv[-1] + current_volume)
        elif float(current_close) < float(previous_close):
            obv.append(obv[-1] - current_volume)
        else:
            obv.append(obv[-1])

    data['OBV'] = pd.Series(obv, index=data.index)  # Ensure OBV aligns with the DataFrame index
    return data

In [ ]:
def calculate_obv_vectorized(data):
    """
    Calculate the On-Balance Volume (OBV) indicator using a vectorized approach.

    Parameters
    ----------
    data : pandas.DataFrame
        A DataFrame containing at least 'Close' and 'Volume' columns with historical price and volume data.

    Returns
    -------
    pandas.DataFrame
        The original DataFrame with an additional 'OBV' column containing the On-Balance Volume values.

    Notes
    -----
    - OBV is a cumulative volume-based momentum indicator.
    - Volume is added when the closing price increases, subtracted when it decreases, and unchanged when flat.
    - This implementation uses vectorized operations for performance and robustness.
    - The first OBV value is initialized to 0.

    Example
    -------
    >>> df = yf.download("MSFT", start="2023-01-01", end="2024-01-01")
    >>> df = calculate_obv_vectorized(df)
    >>> df[['Close', 'Volume', 'OBV']].tail()
    """
    direction = np.sign(data['Close'].diff()).fillna(0)
    obv = (direction * data['Volume']).fillna(0).cumsum()
    data['OBV'] = obv
    return data

In [ ]:
from dataclasses import dataclass

@dataclass
class PlotIndicatorsConfig:
    price_ma_ema: bool = True
    rsi: bool = False
    macd: bool = True
    bollinger: bool = False
    stochastic: bool = True
    atr: bool =False
    obv: bool = True
    volume: bool = True

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from dataclasses import asdict

def plot_selected_indicators_for_single_ticker_analysis(data, ticker, config: PlotIndicatorsConfig = None):
    """
    Plot selected technical indicators for a single stock ticker.

    Parameters
    ----------
    data : pandas.DataFrame
        DataFrame with historical price and indicator columns.
    ticker : str
        Stock ticker symbol.
    config : dict, optional
        Dictionary specifying which indicators to plot. Keys are:
        - 'price_ma_ema', 'rsi', 'macd', 'bollinger', 'stochastic', 'atr', 'obv', 'volume'
        Values should be True or False. Defaults to all True.

    Returns
    -------
    None
    """
    default_config = PlotIndicatorsConfig(
        price_ma_ema = True,
        rsi = False,
        macd = True,
        bollinger = False,
        stochastic = True,
        atr =False,
        obv = True,
        volume = True
    )
    config = config or default_config
    config_dict = asdict(config)
    indicators_to_plot = [key for key, val in config_dict.items() if val]

    plt.figure(figsize=(14, 3.5 * len(indicators_to_plot)))

    plot_idx = 1

    if config.price_ma_ema:
        plt.subplot(len(indicators_to_plot), 1, plot_idx)
        plt.plot(data['Close'], label='Close Price')
        plt.plot(data['MA'], label='MA (20)')
        plt.plot(data['EMA'], label='EMA (20)')
        plt.title(f'{ticker} Price with MA & EMA')
        plt.legend()
        plot_idx += 1

    if config.rsi:
        plt.subplot(len(indicators_to_plot), 1, plot_idx)
        plt.plot(data['RSI'], label='RSI (14)')
        plt.axhline(70, color='r', linestyle='--')
        plt.axhline(30, color='r', linestyle='--')
        plt.title('RSI')
        plt.legend()
        plot_idx += 1

    if config.macd:
        plt.subplot(len(indicators_to_plot), 1, plot_idx)
        plt.plot(data['MACD'], label='MACD')
        plt.plot(data['Signal Line'], label='Signal Line')
        plt.title('MACD')
        plt.legend()
        plot_idx += 1

    if config.bollinger:
        plt.subplot(len(indicators_to_plot), 1, plot_idx)
        plt.plot(data['Close'], label='Close Price')
        plt.plot(data['Upper Band'], label='Upper Band')
        plt.plot(data['Lower Band'], label='Lower Band')
        plt.title('Bollinger Bands')
        plt.legend()
        plot_idx += 1

    if config.stochastic:
        plt.subplot(len(indicators_to_plot), 1, plot_idx)
        plt.plot(data['%K'], label='%K')
        plt.plot(data['%D'], label='%D')
        plt.axhline(80, color='r', linestyle='--')
        plt.axhline(20, color='r', linestyle='--')
        plt.title('Stochastic Oscillator')
        plt.legend()
        plot_idx += 1

    if config.atr:
        plt.subplot(len(indicators_to_plot), 1, plot_idx)
        plt.plot(data['ATR'], label='ATR')
        plt.title('Average True Range')
        plt.legend()
        plot_idx += 1

    if config.obv and 'OBV' in data.columns:
        plt.subplot(len(indicators_to_plot), 1, plot_idx)
        plt.plot(data['OBV'], label='OBV')
        plt.title('On-Balance Volume')
        plt.legend()
        plot_idx += 1

    if config.volume:
        plt.subplot(len(indicators_to_plot), 1, plot_idx)
        plt.plot(data['Volume'], label='Volume')
        plt.title('Volume')
        plt.legend()
        plot_idx += 1

    plt.tight_layout()
    plt.show()

In [ ]:
def plot_indicators_for_single_ticker_analysis(data, ticker):
    """
    Generate a multi-panel chart displaying key technical indicators for a single stock ticker.

    Parameters
    ----------
    data : pandas.DataFrame
        A DataFrame containing historical price and volume data along with precomputed indicator columns.
        Required columns include:
        - 'Close', 'MA', 'EMA'
        - 'RSI'
        - 'MACD', 'Signal Line'
        - 'Upper Band', 'Lower Band'
        - '%K', '%D'
        - 'ATR'
        - 'Volume'
        (Optional: 'OBV' if uncommented in the plotting section.)
    ticker : str
        The stock ticker symbol used for labeling the chart.

    Returns
    -------
    None
        Displays a matplotlib figure with 8 vertically stacked subplots:
        1. Close Price with MA and EMA
        2. RSI with overbought/oversold thresholds
        3. MACD and Signal Line
        4. Bollinger Bands with Close Price
        5. Stochastic Oscillator (%K and %D)
        6. Average True Range (ATR)
        7. (Optional) On-Balance Volume (OBV)
        8. Volume

    Notes
    -----
    - This function is designed for in-depth analysis of a single ticker.
    - Ensure all required indicators are computed and present in the DataFrame before calling this function.
    - The layout uses 8 vertical subplots; adjust `figsize` or subplot count if modifying the indicator set.
    - The OBV panel is currently commented out but can be enabled if desired.

    Example
    -------
    >>> df = fetch_data("AAPL", "2023-01-01", "2024-01-01")
    >>> df = apply_indicators(df)
    >>> plot_indicators(df, "AAPL")
    """

    plt.figure(figsize=(14, 20))

    # Plot Closing Price and Moving Averages
    plt.subplot(8, 1, 1)
    plt.plot(data['Close'], label='Close Price')
    plt.plot(data['MA'], label='MA (20)')
    plt.plot(data['EMA'], label='EMA (20)')
    plt.title(f'{ticker} Stock Price and Moving Averages')
    plt.legend()

    # Plot RSI
    plt.subplot(8, 1, 2)
    plt.plot(data['RSI'], label='RSI (14)')
    plt.axhline(y=70, color='r', linestyle='--')
    plt.axhline(y=30, color='r', linestyle='--')
    plt.title('RSI')
    plt.legend()

    # Plot MACD
    plt.subplot(8, 1, 3)
    plt.plot(data['MACD'], label='MACD (12, 26)')
    plt.plot(data['Signal Line'], label='Signal Line (9)')
    plt.title('MACD')
    plt.legend()

    # Plot Bollinger Bands
    plt.subplot(8, 1, 4)
    plt.plot(data['Close'], label='Close Price')
    plt.plot(data['Upper Band'], label='Upper Band')
    plt.plot(data['Lower Band'], label='Lower Band')
    plt.title('Bollinger Bands')
    plt.legend()

    # Plot Stochastic Oscillator
    plt.subplot(8, 1, 5)
    plt.plot(data['%K'], label='%K')
    plt.plot(data['%D'], label='%D')
    plt.axhline(y=80, color='r', linestyle='--')
    plt.axhline(y=20, color='r', linestyle='--')
    plt.title('Stochastic Oscillator')
    plt.legend()

    # Plot ATR
    plt.subplot(8, 1, 6)
    plt.plot(data['ATR'], label='ATR (14)')
    plt.title('Average True Range (ATR)')
    plt.legend()

    # Plot OBV
    plt.subplot(8, 1, 7)
    plt.plot(data['OBV'], label='OBV')
    plt.title('On-Balance Volume (OBV)')
    plt.legend()

    # Plot Volume 
    plot_volume_as_bar = False  # Change to True to plot as bar chart
    plt.subplot(8, 1, 8) 
    if plot_volume_as_bar:
        data['Volume'].plot(kind='bar', label='Volume', width=1.0) 
        plt.bar(data.index, data['Volume'], label='Volume', width=1.0)
        normalized_volume = data['Volume'] / data['Volume'].max()
        plt.bar(x=mdates.date2num(data.index), height=data['Volume'].max(), data=normalized_volume, label='Volume')
        plt.title('Volume')
    else:
        plt.plot(data['Volume'], label='Volume')
        plt.title('Volume')
        plt.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_multi_tickers_comparison(data_dict, normalized=False):
    """
    Plot selected indicators across multiple tickers on shared subplots
    for comparative analysis.

    Parameters
    ----------
    data_dict : dict
        Dictionary of {ticker: DataFrame} pairs with indicators already applied.
    normalized : bool, optional (default=True)
        Whether to normalize close price and replace volume data with ATR normalized
        for better visual comparison.
    """
    plt.figure(figsize=(16, 20))

    # 1. Close Price - optionally normalized
    plt.subplot(4, 1, 1)
    if not normalized:
        for ticker, df in data_dict.items():
            plt.plot(df['Close'], label=f'{ticker} Close')
        plt.title('Close Price')
    else:   
        for ticker, df in data_dict.items():
            norm_price = df['Close'] / df['Close'].iloc[0]
            plt.plot(norm_price, label=f'{ticker} Price')
        plt.title('Normalized Close Price')
    plt.legend()

    # 2. RSI
    plt.subplot(4, 1, 2)
    for ticker, df in data_dict.items():
        plt.plot(df['RSI'], label=f'{ticker} RSI')
    plt.axhline(70, color='r', linestyle='--')
    plt.axhline(30, color='r', linestyle='--')
    plt.title('RSI')
    plt.legend()

    # 3. MACD
    plt.subplot(4, 1, 3)
    for ticker, df in data_dict.items():
        plt.plot(df['MACD'], label=f'{ticker} MACD')
    plt.title('MACD')
    plt.legend()

    # 4. Volume - optionally replaced with ATR normalized
    if normalized:
        # ATR normalized for scale
        plt.subplot(4, 1, 4)
        for ticker, df in data_dict.items():
            norm_atr = df['ATR'] / df['ATR'].max()
            plt.plot(norm_atr, label=f'{ticker} Normalized ATR')
        plt.title('Normalized ATR (as Volume Proxy)')
    else:   
        # Volume 
        plt.subplot(4, 1, 4)
        for ticker, df in data_dict.items():
            plt.plot(df['Volume'], label=f'{ticker} Volume')
        plt.title('Volume')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
def calculate_indicators(data):
    data = calculate_ma(data)
    data = calculate_ema(data)
    data = calculate_rsi(data)
    data = calculate_macd(data)
    data = calculate_bollinger_bands(data)
    data = calculate_stochastic_oscillator(data)
    data = calculate_atr(data)
    data = calculate_obv_vectorized(data)
    # data = calculate_obv_guard(data)
    return data

In [ ]:
# Example usage
ticker = 'BABA' 
start_date = '2022-01-01'
end_date = '2025-11-30'

data = fetch_yfinance_data(ticker, start_date, end_date)
data = calculate_indicators(data)
# plot indicators for each ticker individually
plot_selected_indicators_for_single_ticker_analysis(data, ticker)

In [ ]:
plot_indicators_for_single_ticker_analysis(data, ticker, )

In [ ]:
only_price_and_volume_config = PlotIndicatorsConfig(
    price_ma_ema = True,
    rsi = False,
    macd = False,
    bollinger = False,
    stochastic = False,
    atr =False,
    obv = False,
    volume = True
)
plot_selected_indicators_for_single_ticker_analysis(data, ticker, only_price_and_volume_config)

In [ ]:
# Example usage
tickers = ['ROK','NVDA','BABA'] 
start_date = '2022-01-01'
end_date = '2025-11-30'

# load data to a dictionary 
data_dict = {}
for ticker in tickers:
    data = fetch_yfinance_data(ticker, start_date, end_date)
    data = calculate_indicators(data)
    # plot indicators for each ticker individually
    plot_indicators_for_single_ticker_analysis(data, ticker)
    data_dict[ticker] = data

# plot comparative indicators across all tickers
plot_multi_tickers_comparison(data_dict)
plot_multi_tickers_comparison(data_dict, normalized=True)

In [ ]:
# Example usage
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'BABA', 'JD', 'BIDU']
start_date = '2022-01-01'
end_date = '2025-11-30'

# load data to a dictionary 
data_dict = {}
for ticker in tickers:
    data = fetch_yfinance_data(ticker, start_date, end_date)
    data = calculate_indicators(data)
    # plot indicators for each ticker individually
    plot_indicators_for_single_ticker_analysis(data, ticker)
    data_dict[ticker] = data

# plot comparative indicators across all tickers
plot_multi_tickers_comparison(data_dict)
plot_multi_tickers_comparison(data_dict, normalized=True)

### Candlestick and Bollinger bands added

In [ ]:
# data['BB_MID']  # Middle band (usually a moving average)
# data['BB_UPPER']
# data['BB_LOWER']

def calculate_bollinger_bands_updated(df, window=20, num_std=2):
    df = df.copy()
    df['BB_MID'] = df['Close'].rolling(window).mean()
    df['BB_STD'] = df['Close'].rolling(window).std()
    df['BB_UPPER'] = df['BB_MID'] + num_std * df['BB_STD']
    df['BB_LOWER'] = df['BB_MID'] - num_std * df['BB_STD']
    return df

In [ ]:
def prepare_for_mplfinance(data):
    """
    Ensure OHLCV columns are numeric and drop rows with missing values.
    """
    ohlcv_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
    data = data.copy()

    # Convert to numeric (in case of object dtype)
    for col in ohlcv_cols:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Drop rows with any NaNs in OHLCV
    data.dropna(subset=ohlcv_cols, inplace=True)

    return data

In [ ]:
def extract_single_ticker_ohlcv(data, ticker):
    """
    Extract and rename OHLCV columns from a MultiIndex DataFrame for a single ticker.
    """
    ohlcv = data.xs(ticker, axis=1, level='Ticker')
    ohlcv = ohlcv[['Open', 'High', 'Low', 'Close', 'Volume']]  # Drop 'Adj Close'
    ohlcv.dropna(inplace=True)
    return ohlcv

In [ ]:
import mplfinance as mpf

In [ ]:
def make_bollinger_addplots(df):
    return [
        mpf.make_addplot(df['BB_UPPER'], color='blue', width=0.8),
        mpf.make_addplot(df['BB_MID'],   color='gray', linestyle='--', width=0.8),
        mpf.make_addplot(df['BB_LOWER'], color='blue', width=0.8)
    ]

In [ ]:
def plot_candlestick_with_indicators(df, ticker, show_bollinger=True):
    df = df.copy()

    # Flatten MultiIndex if present
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    # Now you can safely filter columns by name
    bb_cols = [col for col in df.columns if col.startswith('BB_')]
    df = df[['Open', 'High', 'Low', 'Close', 'Volume'] + bb_cols]
    df.dropna(inplace=True)

    addplots = []
    if show_bollinger:
        addplots += make_bollinger_addplots(df)

    mpf.plot(df, type='candle', style='charles', volume=True,
             title=f'{ticker} Candlestick Chart',
             figsize=(16, 8), tight_layout=True, xrotation=15,
             addplot=addplots)

In [ ]:
data = fetch_yfinance_data("ROK", "2023-08-01", "2025-11-30", auto_adjust=False)
data = calculate_bollinger_bands_updated(data)
plot_candlestick_with_indicators(data, "ROK", show_bollinger=True)

In [ ]:
def plot_candlestick_chart(ticker, start, end, auto_adjust=True):
    data = yf.download(ticker, start=start, end=end, auto_adjust=False)
    ohlcv = data.xs(ticker, axis=1, level='Ticker')
    ohlcv = ohlcv[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()

    if not auto_adjust:
        mpf.plot(ohlcv, type='candle', style='charles', volume=True, title=f'{ticker} Candlestick Chart')
    else:
        mpf.plot(ohlcv, type='candle', style='charles', volume=True,
         title=f'{ticker} Candlestick Chart',
         figratio=(16, 6))  # Wider chart
        mpf.plot(ohlcv, type='candle', style='charles', volume=True,
         title=f'{ticker} Candlestick Chart',
         figsize=(16, 8))  # Width x Height in inches

In [ ]:
# Example usage
ticker = 'ROK' 
start_date = '2025-10-01'
end_date = '2025-11-30'
plot_candlestick_chart(ticker, start_date, end_date)

In [ ]:


# Example usage
ticker = 'ROK' 
start_date = '2022-01-01'
end_date = '2025-11-30'

data = fetch_yfinance_data(ticker, start_date, end_date, auto_adjust=False)
print(data.columns)
print(data.head())
# data = prepare_for_mplfinance(data)
# ohlcv = extract_single_ticker_ohlcv(data, ticker)
# data = calculate_indicators(data)
# data = fetch_yfinance_data(ticker, start_date, end_date, auto_adjust=False)
ohlcv = extract_single_ticker_ohlcv(data, ticker)
mpf.plot(ohlcv, type='candle', style='charles', volume=True, title=f'{ticker} Candlestick Chart')

    
# mpf.plot(data, type='candle', style='charles', volume=True, title='Candlestick Chart')